# Projekt zur Spurerkennung im Wahlfach "Digitale Bildverarbeitung" 

In diesem Jupyter-Notebook wird eine Fahrbahnmarkierungserkennung implementiert. Anhand von Bildern und Videos von Udacity und KITTI wird diese Erkennung auf verschiedenste Weise getestet.

Um eine Fahrbahn erkennen zu können müssen folgende Schritte während des Programmablaufs abgearbeitet werden:

    - Kamerakalibrierung
    - Perspektivtransformation
    - Maskieren des Bildes mit gelben und weißen Farbmasken
    - "Sliding Windows"
    - Berechnung der Polynome für die Eingrenzung der Fahrbahn
    - Rücktransformation der berechneten Punkte der Polynome
    - Einzeichnen der Fläche zwischen den Polynomen

Im folgenden werden die jeweils verwendeten Funktionen genauer erklärt, am Ende finden sich die Bilder zur Kamerakalibrierung sowie zur Spurerkennung.

<sup>Bearbeitet wurde das Projekt von Alexander Schulte(), Edmund Krain () und Marcel Fleck (9611872)</sup>

In [ ]:
# imports
import cv2 as cv
import numpy as np
import time
from matplotlib import pyplot as plt
import glob
from prettytable import PrettyTable
from deprecated import deprecated

# Time Measurement

Diese Funktionen dienen der Bestimmung von zeitaufwändigen Arbeitsschritten, um eine Verbesserung der Framerate zu erleichtern.

In einem Array wird ein Objekt hinterlegt, in dem der Name, die Start- und Endzeit sowie die Anzahl an Aufrufen hinterlegt wird (siehe ```start_time_measurement``` und ```end_time_measurement```).

Diese Informationen werden dann in einer Tabelle nach Beendigung des Programmdurchlaufs visualisiert (siehe ```analyse_time_measurement```).

Abbildung 1 zeigt ein Beispile für eine solche Tabelle:

<p align="center">
  <img src="img/documentation/timemeasurement.png" height="35%" width="35%" />
  <br>
  <em>Abbildung 1: Tabelle zur Darstellung der Zeitmessungsergebnisse</em>
</p>


In [ ]:
time_measurements = {}

def start_time_measurement(eventName):
    """
    Use this function to start a time measurement for a specific event.
    A end_time_measurement() call with the same name must be called before the next start_time_measurement() call.
    """
    add = False
    if eventName not in time_measurements:
        time_measurements[eventName] = {
            "name": eventName,
            "start": [],
            "end": [],
            "count": 0
        }
        add = True
    elif len(time_measurements[eventName]["start"]) > len(time_measurements[eventName]["end"]):
        print(f"Time measure error: Event '{eventName}' not finished before reassignment!")
    else:
       add = True
    
    #start measurement as late as possible
    if add:
        time_measurements[eventName]["start"].append(time.perf_counter_ns())

def end_time_measurement(eventName):
    """
    Use this function to end a time measurement for a specific event.
    A time measurement with the same name must be started with start_time_measurement() before it can be ended.
    """
    #end measurement as fast as possible
    temp_time = time.perf_counter_ns()
    if eventName not in time_measurements:
        print(f"Time measure error: Event '{eventName}' not defined!")
    elif len(time_measurements[eventName]["end"]) >= len(time_measurements[eventName]["start"]):
        print(f"Time measure error: Event '{eventName}' not started before reassignment!")
    else:
        time_measurements[eventName]["end"].append(temp_time)
        time_measurements[eventName]["count"] += 1
    
def analyse_time_measurements():
    """
    Analyse time measurements and print them in a table
    """
    time_measurements_table = PrettyTable(["Name", "Avg. [ms]", "Min. [ms]", "Max. [ms]", "Occurrences [compl.]"])
    time_measurements_table.align["Name"] = "l"
    time_measurements_table.align["Avg. [ms]"] = "r"
    time_measurements_table.align["Min. [ms]"] = "r"
    time_measurements_table.align["Max. [ms]"] = "r"
    time_measurements_table.align["Occurrences [compl.]"] = "r"
    for key, event in time_measurements.items():
        timings = []
        if len(event["start"]) != len(event["end"]):
            print(f"Time measure error: Event '{key}' has different amounts of values for start and end times!")
        else:
            #exclude 0 values
            for i in range(len(event["start"])):
                timing = (event["end"][i] - event["start"][i])
                if timing >= 0:
                    timing = timing / (1000 * 1000) #convert from ns to ms
                    timings.append(timing)

            event["min"] = min(timings) 
            event["max"] = max(timings)
            event["avg"] = sum(timings) / len(event["start"])

            time_measurements_table.add_row([key, '{0:.2f}'.format(event["avg"]), '{0:.2f}'.format(event["min"]), '{0:.2f}'.format(event["max"]), event["count"]])
    print(time_measurements_table)

# Kamerakalibrierung

Die Abbildung eines 3D-Objekts auf eine 2D-Bildebene mit einer Kamera wird durch die internen Größen einer Kamera beeinflusst. Dazu gehören zum Beispiel die Bildmitte, die Brennweite und auch Kameraverzerrungsparameter.

Mit Hilfe der Kamerakalibrierung werden die sogenannten "intrinsische" und "extrinsische" Parameter berechnet. Intrinsische Parameter beschreiben die Kalibriermatrix (Kamera zu Pixel), während die extrinsischen Parameter die Beziehung zwischen dem Koordinatensystem der Kamera und dem Welt-Koordinatensystem beschreiben (Welt zu Kamera).

Zusammen bilden diese Parameter eine Projektionsmatrix, die es ermöglicht, gemachte Bilder im Welt-Koordinatensystem darzustellen (die Bilder werden entzerrt). 

In folgendem Abschnitt wird die Kamera mit Hilfe von Schachbrett-Bildern kalibriert. Diese Aufgabe mit den Ergebnissen auf den Schachbrettbildern befinden sich im JupyterNotebook [Projekt_Spurerkennung_v2.jpynb](Projekt_Spurerkennung_v2.ipynb).


Mit den unten definierten Funktionen (siehe ```undistort_image``` und ```undistort_image_remap```) werden dann die Frames der Videos anhand der berechneten Transformations-Matrix entzerrt, um eine tatsächliche Darstellung im Welt-Koordinatensystem zu erhalten. Eine Implementierung von zwei Funktionen lässt mit dem Erreichen von einer ungefähr 50% schnelleren Verarbeitung begründen.

In [ ]:
# size of chessboard, minimum error with (7, 6), but there were severe artefacts at the borders (see error calculation at the end)
chessboard_x, chessboard_y = 9, 6

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
object_points = np.zeros((chessboard_y * chessboard_x, 3), np.float32)
object_points[:, :2] = np.mgrid[0:chessboard_x, 0:chessboard_y].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

# use all calibration images
images = glob.glob("./img/Udacity/calib/*.jpg")
for i, frame_name in enumerate(images):
    image = cv.imread(frame_name)
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(image_gray, (chessboard_x, chessboard_y), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(object_points)
        corners2 = cv.cornerSubPix(
            image_gray, corners, (11, 11), (-1, -1), criteria
        )  # improve accuracy of corners
        imgpoints.append(corners)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objpoints, imgpoints, image_gray.shape[::-1], None, None
)

image_height, image_width = image.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (image_width, image_height), 1, (image_width, image_height))


# ------- define functions for image processing -------
def undistort_image(img):
    image_height, image_width = img.shape[:2]

    img_undist = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, image_width, image_height = roi
    return img_undist[y : y + image_height, x : x + image_width]

# ~50% faster than undistort_image()
def undistort_image_remap(img):
    image_height, image_width = img.shape[:2]
    mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (image_width, image_height), 5)
    dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
    # crop the image
    x, y, image_width, image_height = roi
    return dst[y : y + image_height, x : x + image_width]

# Perspektivtransformation

Die Perspektivtransformation wird verwendet, um die "Region of Interest" festzulegen. Dadurch muss nicht mehr das gesamte Bild verarbeitet werden, sondern es kann nur ein der Bereich, in dem sich die Fahrspur befindet, zur Verarbeitung genutzt werden.

Hierbei werden vier feste Punkte im originalen Bild ausgewählt. Diese Punkte werden dann in einem Bild mit den selben Dimensionen wie das Originalbild dargestellt, um eine rechteckige Beziehung zwischen den Punkten herzustellen (siehe Abbildung 1). Die Transformation geschieht mit der Funktion ```warp_image_udacity```.

<p align="center">
  <img src="img/documentation/perspectiveTransformation.png" />
  <br>
  <em>Abbildung 2: Beispiel für implementierte Perspektivtransformation</em>
</p>

Um die mit einer später erklärten Funktion (```sliding_windows```) in dem transformierten Bild gefundenen Punkte in dem originalen Bild anzeigen zu können, müssen diese Punkte zurücktransformiert werden. Um Rechenleistung zu sparen, werden tatsächlich nur die Punkte transformiert (siehe ```rewarp_points_udacity```), bei einem ersten Ansatz wurde das gesamte Bild zurücktransformiert (siehe ```rewarp_image_udacity```). 






In [ ]:
# udacity images
src_udacity = np.float32([[191, 628], [531, 404], [1021, 628], [681, 404]])
dst_udacity = np.float32([[150, 720], [150, 10], [1000, 720], [1000, 10]])
M_warp = cv.getPerspectiveTransform(src_udacity, dst_udacity)
M_rewarp = cv.getPerspectiveTransform(dst_udacity, src_udacity)


def warp_image_udacity(img):
    img = cv.warpPerspective(img, M_warp, (img.shape[1], img.shape[0]))
    # image = cv.resize(
    #     image, (int(img.shape[1] / 2), int(img.shape[0] / 2))
    # )  # resize to half size
    return img

@deprecated
def rewarp_image_udacity(img):
    # image = cv.resize(
    #     img, (int(img.shape[1] * 2), int(img.shape[0] * 2))
    # ) 
    # img = cv.warpPerspective(img, M_rewarp, (img.shape[1], img.shape[0]))
    img = cv.warpPerspective(img, M_rewarp, (img.shape[1], img.shape[0]), cv.WARP_INVERSE_MAP)
    return img

def rewarp_points_udacity(points):
    return cv.perspectiveTransform(points, M_rewarp)

# Sliding Windows

Die Erkennung der Fahrbahnmarkierungen wird mit den sogenannten "Sliding Windows" umgesetzt. 

Kurz gefasst sind "Sliding Windows" kleine Kästchen, die anhand der Helligkeit von Pixel ihren Mittelpunkt anpassen. Damit kann beispielsweise eine weiße Linie auf schwarzem Hintergrund fast perfekt verfolgt werden. 

Detailreicher beginnt die ```sliding_windows``` Funktion damit, anhand des Histogramms eines Frames die initialen Mittelpunkte der ersten Windows festzulegen. Anhand dieser Mittelpunkte werden eine angegebene Anzahl an Windows erzeugt, die aufeinander gestapelt werden. Das bedeutet, der X-Wert der Windows bleibt gleich, der Y-Wert passt sich an.

Für jedes Window wird nun geprüft, wie viele hellen Pixel von dem jeweiligen Window eingegrenzt werden. Wenn der Wert der gefundenen Pixel das angegebene Minimum unterschreitet, wird der Mittelpunkt des jeweiligen Windows angepasst.

Ein Beispiel für diese "Sliding Windows" zeigt Abbildung 3.

<p align="center">
  <img src="img/documentation/slidingWindows.png" width="40%" height="40%"/>
  <br>
  <em>Abbildung 3: Beispiel für die Darstellung der "Sliding Windows"</em>
</p>

Damit kann die Fahrspur bei einer Veränderung leicht weiterverfolgt und markiert werden. Die hier gefundenen Mittelpunkte bilden die Grundlage für die Polynomberechnung.

In [ ]:
nwindows = 50
margin = 50
def sliding_windows(frame, window_width=200, minimum_whites=30, show_windows=False):
    # Histogram for image
    hist = np.sum(frame[frame.shape[0]//2:, :], axis=0)
        
    # Take peaks from left and right side of histogramm for starting points and add half margin
    mid = np.int32(hist.shape[0] // 2)
    leftx_start = np.argmax(hist[:mid]) - window_width // 2
    rightx_start = np.argmax(hist[mid:]) + mid + window_width // 2
    # Window height based on number of windows
    window_height = np.int32(frame.shape[0] // nwindows)
    
    # Calc points that are not zero in images
    nonzero = frame.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Initialize current positions for windows
    leftx_current = leftx_start
    rightx_current = rightx_start

    # Initialize values to be returned -> centers of windows
    lefts_good = np.empty((0,2), dtype=np.int32)
    rights_good = np.empty((0,2), dtype=np.int32)

    # Go through every window
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = frame.shape[0] - (window + 1) * window_height
        win_y_high = frame.shape[0] - window*window_height
        y_mid = (win_y_low + win_y_high) // 2
        
        # Calculate boundaries of the window
        win_xleft_low = leftx_current - window_width  
        win_xleft_high = leftx_current + window_width  
        win_xright_low =  rightx_current - window_width 
        win_xright_high = rightx_current + window_width  
        
        # Identify the pixels that are not zero within window
        left_inds = ((nonzeroy >= win_y_low ) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        right_inds = ((nonzeroy >= win_y_low ) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # If more than minimum pixels are found -> recenter next window
        if len(left_inds) > minimum_whites:
            leftx_current = np.int32(np.mean(nonzerox[left_inds]))
            lefts_good = np.concatenate((lefts_good, [[leftx_current, y_mid]]))
            if show_windows:
                cv.rectangle(frame, (leftx_current - margin, win_y_low),(leftx_current + margin, win_y_high),(255, 255, 255), 2)
        if len(right_inds) > minimum_whites:
            rightx_current = np.int32(np.mean(nonzerox[right_inds]))
            rights_good = np.concatenate((rights_good, [[rightx_current, y_mid]]))
            if show_windows:
                cv.rectangle(frame, (rightx_current - margin, win_y_low),(rightx_current + margin, win_y_high),(255, 255, 255), 2)

    return mid, lefts_good, rights_good

# Polynome

Um eine Veränderung der Fahrspur, z.B. bei Kurven, angemessen darstellen zu können, werden anhand der von ```sliding_windows``` gefundenen Punkte Polynome berechnet.

Diese bilden dann die rechten und linken Grenzen der einzufärbenden Fläche. Dadurch lassen sich selbst kleine Änderungend er Fahrspur ordentlich darstellen. (siehe Abbildung 4)

<p align="center">
  <img src="img/documentation/polynoms.png" width="40%" height="40%"/>
  <br>
  <em>Abbildung 4: Darstellung der berechneten Polynome auf der Fahrspur</em>
</p>

In [ ]:
max_polynom_curvature_value = 0.005 # defines how much the polynom can be curved before it is considered faulty

def calculate_polynomial_points(lefts, rights):
    """
    calculate polynomial from rewarped points and then return list of points on the polynomial
    @return: array of points for left and right polynomial
    """
    # flags for faulty polynom sides
    is_left_faulty, is_right_faulty = False, False
    
    # check if there are enough points
    if len(lefts) < 2 or len(rights) < 2:
        # print("Not enough points to calculate polynomial")
        return None, None
    
    # calculate polynomial from rewarped points
    left_polynom_values = np.polyfit(lefts[:,1], lefts[:,0], 2)
    right_polynom_values = np.polyfit(rights[:,1], rights[:,0], 2)

    # check if polynom is valid (not too steeply curved)
    if (left_polynom_values[0] > max_polynom_curvature_value or left_polynom_values[0] < -max_polynom_curvature_value):
        is_left_faulty = True
    if (right_polynom_values[0] > max_polynom_curvature_value or right_polynom_values[0] < -max_polynom_curvature_value):
        is_right_faulty = True
    if is_left_faulty and is_right_faulty:
        return None, None
    
    # 750 as x length of polynom
    x_axis = np.linspace(0, 750, 750)

    # calculate y values for left and right line
    left_line_y = left_polynom_values[0]*x_axis**2 + left_polynom_values[1]*x_axis + left_polynom_values[2]
    right_line_y = right_polynom_values[0]*x_axis**2 + right_polynom_values[1]*x_axis + right_polynom_values[2]

    # array of points for left and right line from x and y values
    left_pts = np.array([np.transpose(np.vstack([left_line_y, x_axis]))])
    right_pts = np.array([np.transpose(np.vstack([right_line_y, x_axis]))])
    
    # loop through all points and check if they are to close or to far away from each other
    # (if they are to close -> most likely a interception point)
    for i in range(len(left_pts[0])):
        y_distance = abs(left_pts[0][i][0] - right_pts[0][i][0])
        if y_distance < 10 or y_distance > 1000:
            return None, None

    # only return non faulty polynom points
    return left_pts if not is_left_faulty else None, right_pts if not is_right_faulty else None

# Enzeichen der Fahrspur

Mit Hilfe der Funktion ```drawRecOnFrame``` wird dann die Fläche zwischen den Polynomen eingezeichnet (siehe Abbildung 5).

<p align="center">
  <img src="img/documentation/filledLane.png" width="40%" height="40%"/>
  <br>
  <em>Abbildung 5: Markierung der Fahrbahn mit Polynomen als Grenzen</em>
</p>

Um die Spurerkennung stabiler zu gestalten, werden bei einer zu geringen Anzahl an gefundenen Punkten keine neue Polynome berechnet. Die Spur wird dann anhand der zuletzt berechneten Polynome eingezeichnet.

In [ ]:
def drawRecOnFrame(frame, left_pts, right_pts):
    """
    input: array of points from left and right lane marking 
     e.g.:
      [[ 281   39]
      [ 971  163]
      [ 958  101]]
    """
    if len(left_pts) + len(right_pts) > 3:
      borderPoints = np.concatenate((np.flip(left_pts, axis=0), right_pts))
      borderPointsRewaped = rewarp_points_udacity(np.array([borderPoints], dtype=np.float32))
      borderPointsRewapedInt = borderPointsRewaped.astype(int)
      # draw in frame
      cv.drawContours(frame, borderPointsRewapedInt, -1, (0,255,0), -1)

# Masking

Mit ```applyMasks``` werden bestimmte Farbmasken, genauer eine für Weiß und eine für Gelb, auf das Bild gelegt. Damit werden nur diese zwei Farben in den Bildern hervorgehoben bzw. dargestellt. 

Diese Funktion ist inzwischen veraltet, da die Funktion ```lane_detection``` eine verbesserte Implementierung mit neuen Filtern zur Verfügung stellt.

In [ ]:
@deprecated
def applyMasks(frame):
    """
    apply masks to frame and return frame with only lane marking
    """
    ## convert to hsv
    hls_frame = cv.cvtColor(frame, cv.COLOR_BGR2HLS)

    ## mask for white
    white_mask = cv.inRange(hls_frame, (0, 200, 0), (255, 255,255))

    ## mask for yellow
    # yellow_mask = cv.inRange(hls_frame, (20,90,200), (26, 255, 255))
    yellow_mask = cv.inRange(hls_frame, (10,0,100), (40, 255, 255))

    ## final mask and masked
    mask = cv.bitwise_or(white_mask, yellow_mask)
    frame = cv.bitwise_and(frame,frame, mask=mask)

    return frame

In [ ]:
def lane_detection(frame, flag=False):
    frame = cv.GaussianBlur(frame, (5, 5), 0)
    frame_hls = cv.cvtColor(frame, cv.COLOR_BGR2HLS)
    white_lower = (0, 180, 0)
    white_upper = (254, 254, 254)
    if flag:
        white_lower = (0, 180, 0)
        white_upper = (240, 240, 240)

    white_mask = cv.inRange(frame_hls, white_lower, white_upper)

    frame_lab = cv.cvtColor(frame, cv.COLOR_BGR2LAB)
    yellow_mask = cv.inRange(frame_lab, (150, 100, 140), (255, 140, 200))

    combined_mask = cv.bitwise_or(white_mask, yellow_mask)

    redImg = np.zeros(frame.shape, frame.dtype)
    redImg[:,:] = (0, 0, 255)
    redMask = cv.bitwise_and(redImg, redImg, mask=yellow_mask)

    greenImg = np.zeros(frame.shape, frame.dtype)
    greenImg[:,:] = (0, 255, 0)
    greenMask = cv.bitwise_and(greenImg, greenImg, mask=white_mask)

    masks = redMask + greenMask
    # if not flag:
    #     frame_luv = cv.cvtColor(frame, cv.COLOR_BGR2LUV)
    #     yellow_mask2 = cv.inRange(frame_luv, (150, 100, 150), (255, 140, 200))
    #     combined_yellow = cv.bitwise_and(yellow_mask, yellow_mask2)
    #     combined_mask = cv.bitwise_or(white_mask, combined_yellow)
    #     blueImg = np.zeros(frame.shape, frame.dtype)
    #     blueImg[:,:] = (255, 0, 0)
    #     blueMask = cv.bitwise_and(blueImg, blueImg, mask=yellow_mask2)
    #     masks = masks + blueMask

    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 3))
    lanes = cv.morphologyEx(frame_hls[:, :, 1], cv.MORPH_TOPHAT, kernel)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (13, 13))
    lanes_yellow = cv.morphologyEx(frame_lab[:, :, 2], cv.MORPH_TOPHAT, kernel)

    ret, lanes_yellow = cv.threshold(lanes_yellow, thresh=2, maxval=255, type=cv.THRESH_BINARY)
    lanes_yellow = cv.morphologyEx(lanes_yellow, cv.MORPH_OPEN, np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8'), iterations=4)

    lanes = cv.morphologyEx(frame_hls[:, :, 1], 
                              cv.MORPH_BLACKHAT,
                              kernel)

    ret, lanes = cv.threshold(lanes, thresh=5, maxval=255, type=cv.THRESH_BINARY)

    combined_mask = cv.bitwise_or(combined_mask, lanes_yellow)

    combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_OPEN, np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8'), iterations=2)
    # combined_mask = cv.Canny(combined_mask, 50, 150)
    frame = cv.bitwise_and(frame, frame, mask=combined_mask)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    cv.imshow("Masks", masks)
    cv.imshow("TOP_HAT_LANES_YELLOW", lanes_yellow)
    cv.imshow("BLACK_HAT_LANES", lanes)
    cv.imshow("Final", combined_mask)
    return frame

# Helligkeitskorrektur

Damit die Fahrbahnmarkierungen auch in dunklen Elementen der Videos zu erkennen sind, wurde eine dynamische Helligkeitsanpassung implementiert.

Das Bild wird in HSV eingelesen und in die jeweiligen Bestandteile aufgeteilt. Wenn der Helligkeitswert des Frames dann kleiner oder größer ist als die angegebenen Grenzen, dann wird die Helligkeit des Bilds so angepasst, dass sie sich dann zwischen den gesetzten Grenzen befindet (vgl. Abbildung 5).

<p align="middle" float="left">
  <img src="img/documentation/brightnessCorrectionNormal.png" width="40%" height="40%"/>
  <img src="img/documentation/brightnessCorrectionCorrected.png" width="40%" height="40%"/>
  <br>
  <em>Abbildung 5: Helligkeitskorrektur - Links: Original, Rechts: Korrigiert</em>
</p>



In [ ]:
def correct_Brightness(frame):
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    brightness = frame_hsv[...,2].mean()
    brightness_upper_limit = 115
    brightness_lower_limit = 85
    flag = False
    if brightness > brightness_upper_limit:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        h, s, v = cv.split(hsv)
        
        lim = round(0 + brightness - brightness_upper_limit)
        v[v < lim] = 0
        v[v >= lim] -= lim

        final_hsv = cv.merge((h, s, v))
        frame = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
        flag = True
    
    if brightness < brightness_lower_limit:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        h, s, v = cv.split(hsv)
        
        lim = round(255 + brightness_lower_limit - brightness)
        v[v > lim] = 255
        v[v <= lim] += lim

        final_hsv = cv.merge((h, s, v))
        frame = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
        flag = False

    return frame, flag, brightness

# Main

Folgender Abschnitt zeigt den gesamten Ablauf der Fahrspurerkennung.

In Zeile 2 lässt sich festlegen, in welchem der gegebenen Videos die Spurerkennung durchgeführt werden soll. Zur Auswahl stehen: "project", "challenge" und "harder_challenge".

Nach einem Durchlauf des Programms findet sich am Ende des Notebooks die oben angesprochene Tabelle, die den zeitlichen Anteil der einzelnen Funktionen zeigt.

In [ ]:
# Open video file
# video_file = "project"
video_file = "challenge"
# video_file = "harder_challenge"
capture = cv.VideoCapture("./img/Udacity/" + video_file + "_video.mp4")

# Check if camera opened successfully
if capture.isOpened() == False:
    print("Error opening video stream or file")

# Start timer for fps counter
start_timer = time.time() - 0.01
frame_count = -1

# cache between frames
old_left_pts = None
old_right_pts = None

# reset time analysis
time_measurements = {}

# Read every frame
while capture.isOpened():
    ret, frame = capture.read()

    # Check if there is another frame
    if frame is None:
        break

    # Calculate Frame rate
    frame_count += 1
    elapsed_time = time.time() - start_timer
    frame_rate = frame_count / elapsed_time
    
    if ret == True:
        start_time_measurement("frame")
        
        # ----------- Preprocessing ---------------
        start_time_measurement("Preprocessing")
        frame = undistort_image_remap(frame)
        frame_undistorted = frame.copy()
        frame = warp_image_udacity(frame)
        end_time_measurement("Preprocessing")
        
        # ---------  Masking ---------------------
        start_time_measurement("masking")
        # frame = applyMasks(frame)
        frame, flag, brightness = correct_Brightness(frame)
        grayscale_frame = lane_detection(frame, flag)
        end_time_measurement("masking")

        #---------- Sliding Windows ----------
        start_time_measurement("sliding windows")
        # Convert to grayscale for sliding windows
        midpoint, lefts, rights = sliding_windows(grayscale_frame, minimum_whites=margin, show_windows=False)
        end_time_measurement("sliding windows")

        #---------- Calculate polynomial ----------
        start_time_measurement("polynomial calculation")
        left_pts, right_pts = calculate_polynomial_points(lefts, rights)
        end_time_measurement("polynomial calculation")

        # -------- draw plane from all sliding windows ------------
        start_time_measurement("draw plane")
        if left_pts is not None:
            old_left_pts = left_pts        
        if right_pts is not None:
            old_right_pts = right_pts
        drawRecOnFrame(frame_undistorted, old_left_pts[0], old_right_pts[0])
        if old_left_pts is  None and old_right_pts is  None:
            print("No plane drawn")
        end_time_measurement("draw plane")

        # -------- Add frame rate to video ------------
        cv.putText(frame_undistorted, "FPS: " + str(round(frame_rate)), (0, 25),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.putText(frame_undistorted, "Frame: " + str(frame_count), (0, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.putText(frame_undistorted, "Brightness: " + str(np.round(brightness)), (0, 75), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.imshow("Frame", frame_undistorted)


        end_time_measurement("frame")
        # Close video with letter 'q'
        if cv.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# When everything done, release the video capture object
capture.release()
cv.destroyAllWindows()

# create time measurement analysis and print out results
analyse_time_measurements()